In [1]:
# Step 1: Data Preprocessing
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import pandas as pd

import tensorflow as tf
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import (Input, Dense, Dropout, 
                                     Flatten, Conv2D, GlobalAveragePooling2D,
                                     MaxPooling2D, BatchNormalization)
from tensorflow.keras.optimizers import Adam, RMSprop, Adamax
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
train_data_dir = 'C:/Users/danie/sign-language-alpha/data/asl_alphabet_train/asl_alphabet_train'

def generate_dataset(path):
    data = {'path': [], 'label': []}
    
    folders = os.listdir(path)
    
    for folder in folders:
        folderpath = os.path.join(path, folder)
        files = os.listdir(folderpath)
        
        for file in files:
            filepath = os.path.join(folderpath, file)
            
            data['path'].append(filepath)
            data['label'].append(folder)
    
    return pd.DataFrame(data)

df = generate_dataset(train_data_dir)
df.head(10)

,path,label
0,C:/Users/danie/sign-language-alpha/data/asl_al...,A
1,C:/Users/danie/sign-language-alpha/data/asl_al...,A
2,C:/Users/danie/sign-language-alpha/data/asl_al...,A
3,C:/Users/danie/sign-language-alpha/data/asl_al...,A
4,C:/Users/danie/sign-language-alpha/data/asl_al...,A
5,C:/Users/danie/sign-language-alpha/data/asl_al...,A
6,C:/Users/danie/sign-language-alpha/data/asl_al...,A
7,C:/Users/danie/sign-language-alpha/data/asl_al...,A
8,C:/Users/danie/sign-language-alpha/data/asl_al...,A
9,C:/Users/danie/sign-language-alpha/data/asl_al...,A


In [3]:
def dataset_splitter(dataset, train_size = 0.9, shuffle = True, random_state = 0):
    train_df, val_df = train_test_split(dataset, train_size = train_size, shuffle = shuffle, random_state = random_state)

    train_df = train_df.reset_index(drop = True)
    val_df = val_df.reset_index(drop = True)
    
    return train_df, val_df

train_df, val_df = dataset_splitter(df)

In [4]:
BATCH_SIZE = 128
IMG_SIZE = (64, 64)

train_generator = ImageDataGenerator(1./255, 
                                     rotation_range = 10, 
                                     height_shift_range = 0.1, 
                                     width_shift_range = 0.1)

val_generator = ImageDataGenerator(1./255)

train_images = train_generator.flow_from_dataframe(train_df, x_col = 'path', y_col = 'label', 
                                                   color_mode = 'grayscale', class_mode = 'categorical',
                                                   batch_size = BATCH_SIZE, target_size = IMG_SIZE, 
                                                   shuffle = True, seed = 0)

val_images = val_generator.flow_from_dataframe(val_df, x_col = 'path', y_col = 'label', 
                                               color_mode = 'grayscale', class_mode = 'categorical', 
                                               batch_size = BATCH_SIZE, target_size = IMG_SIZE)

Found 74798 validated image filenames belonging to 29 classes.


C:\Users\danie\anaconda3\envs\myenv\lib\site-packages\keras\src\legacy\preprocessing\image.py:920: UserWarning: Found 1 invalid image filename(s) in x_col="path". These filename(s) will be ignored.
  warnings.warn(


Found 8312 validated image filenames belonging to 29 classes.


In [5]:
import cv2
import numpy as np
from keras.models import load_model

model = load_model("FINAL.h5")

class_mapping = train_images.class_indices

def get_class_label(predictions, class_mapping):
    labels_mapping = {v: k for k, v in class_mapping.items()}
    predicted_labels = [labels_mapping[pred] for pred in predictions]
    return predicted_labels


In [6]:
import os

train_images = train_generator.flow_from_dataframe(train_df, x_col = 'path', y_col = 'label', 
                                                   color_mode = 'grayscale', class_mode = 'categorical',
                                                   batch_size = BATCH_SIZE, target_size = IMG_SIZE, 
                                                   shuffle = True, seed = 0)

imgs_dir = r'C:\Users\danie\sign-language-alpha\data\asl_alphabet_test\asl_alphabet_test'
imgs = os.listdir(imgs_dir)

class_mapping = train_images.class_indices

def get_class_label(predictions, class_mapping):
    labels_mapping = {v: k for k, v in class_mapping.items()}
    predicted_labels = [labels_mapping[pred] for pred in predictions]

    return predicted_labels

def predict_image_class(model, img_path, class_mapping):
    img = cv2.imread(os.path.join(imgs_dir, img_path))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = np.expand_dims(cv2.resize(img, (64, 64)), axis = 0)
    predictions = np.argmax(model.predict(img, verbose = 0), axis = 1)
    predicted_labels = get_class_label(predictions, class_mapping)
    return predicted_labels

import cv2
import numpy as np

def process_and_save_video_frame(frame, save_path="processed_frame.jpg"):
    """
    Process a video frame: convert to grayscale, resize with aspect ratio,
    normalize, reshape, and save the processed frame.
    
    Args:
        frame: The original video frame captured from the webcam.
        save_path: The path to save the processed frame image.
    
    Returns:
        frame_final: The processed frame ready for model input.
    """
    def resize_with_aspect_ratio(image, target_size):
        h, w = image.shape[:2]
        target_w, target_h = target_size
        scale = min(target_w / w, target_h / h)
        new_w, new_h = int(w * scale), int(h * scale)
        resized = cv2.resize(image, (new_w, new_h))
        delta_w, delta_h = target_w - new_w, target_h - new_h
        top, bottom = delta_h // 2, delta_h - (delta_h // 2)
        left, right = delta_w // 2, delta_w - (delta_w // 2)
        color = [0, 0, 0]
        new_image = cv2.copyMakeBorder(resized, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
        return new_image

    print(f"Original Frame shape: {frame.shape}")
    
    # Convert to grayscale
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Resize with aspect ratio
    frame_resized = resize_with_aspect_ratio(frame_gray, (64, 64))
    print(f"Shape after resizing with aspect ratio to 64x64: {frame_resized.shape}")
    
    # Normalize
    frame_normalized = frame_resized.astype('float32') / 255.0
    
    # Add batch and channel dimensions
    frame_final = np.expand_dims(frame_normalized, axis=0)
    frame_final = np.expand_dims(frame_final, axis=-1)
    print(f"Shape after adding batch and channel dimensions: {frame_final.shape}")
    
    # Save the processed frame
    cv2.imwrite(save_path, frame_resized * 255)
    
    return frame_final


for img in imgs:
    predicted_labels = predict_image_class(model, img, class_mapping)
    print(f'Predicted Labels {img} -----> {predicted_labels}')

Found 74798 validated image filenames belonging to 29 classes.


C:\Users\danie\anaconda3\envs\myenv\lib\site-packages\keras\src\legacy\preprocessing\image.py:920: UserWarning: Found 1 invalid image filename(s) in x_col="path". These filename(s) will be ignored.
  warnings.warn(


Predicted Labels A_test.jpg -----> ['A']
Predicted Labels B_test.jpg -----> ['B']
Predicted Labels C_test.jpg -----> ['C']
Predicted Labels D_test.jpg -----> ['D']
Predicted Labels E_test.jpg -----> ['E']
Predicted Labels F_test.jpg -----> ['F']
Predicted Labels G_test.jpg -----> ['G']
Predicted Labels H_test.jpg -----> ['H']
Predicted Labels I_test.jpg -----> ['I']
Predicted Labels J_test.jpg -----> ['J']
Predicted Labels K_test.jpg -----> ['K']
Predicted Labels L_test.jpg -----> ['L']
Predicted Labels M_test.jpg -----> ['M']
Predicted Labels nothing_test.jpg -----> ['nothing']
Predicted Labels N_test.jpg -----> ['N']
Predicted Labels O_test.jpg -----> ['O']
Predicted Labels P_test.jpg -----> ['P']
Predicted Labels Q_test.jpg -----> ['Q']
Predicted Labels R_test.jpg -----> ['R']
Predicted Labels space_test.jpg -----> ['space']
Predicted Labels S_test.jpg -----> ['S']
Predicted Labels T_test.jpg -----> ['T']
Predicted Labels U_test.jpg -----> ['U']
Predicted Labels V_test.jpg -----> ['

In [7]:
imgs_dir = r'C:\Users\danie\sign-language-alpha\data\asl_alphabet_test\asl_alphabet_test'
imgs = os.listdir(imgs_dir)

class_mapping = train_images.class_indices

def get_class_label(predictions, class_mapping):
    labels_mapping = {v: k for k, v in class_mapping.items()}
    predicted_labels = [labels_mapping[pred] for pred in predictions]

    return predicted_labels

def predict_image_class(model, img_path, class_mapping):
    img = cv2.imread(os.path.join(imgs_dir, img_path))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = np.expand_dims(cv2.resize(img, (64, 64)), axis = 0)
    predictions = np.argmax(model.predict(img, verbose = 0), axis = 1)
    predicted_labels = get_class_label(predictions, class_mapping)
    return predicted_labels

import cv2
import numpy as np

def process_and_save_video_frame(frame, save_path="processed_frame.jpg"):
    """
    Process a video frame: convert to grayscale, resize with aspect ratio,
    normalize, reshape, and save the processed frame.
    
    Args:
        frame: The original video frame captured from the webcam.
        save_path: The path to save the processed frame image.
    
    Returns:
        frame_final: The processed frame ready for model input.
    """
    def resize_with_aspect_ratio(image, target_size):
        h, w = image.shape[:2]
        target_w, target_h = target_size
        scale = min(target_w / w, target_h / h)
        new_w, new_h = int(w * scale), int(h * scale)
        resized = cv2.resize(image, (new_w, new_h))
        delta_w, delta_h = target_w - new_w, target_h - new_h
        top, bottom = delta_h // 2, delta_h - (delta_h // 2)
        left, right = delta_w // 2, delta_w - (delta_w // 2)
        color = [0, 0, 0]
        new_image = cv2.copyMakeBorder(resized, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
        return new_image

    print(f"Original Frame shape: {frame.shape}")
    
    # Convert to grayscale
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Resize with aspect ratio
    frame_resized = resize_with_aspect_ratio(frame_gray, (64, 64))
    print(f"Shape after resizing with aspect ratio to 64x64: {frame_resized.shape}")
    
    # Normalize
    frame_normalized = frame_resized.astype('float32') / 255.0
    
    # Add batch and channel dimensions
    frame_final = np.expand_dims(frame_normalized, axis=0)
    frame_final = np.expand_dims(frame_final, axis=-1)
    print(f"Shape after adding batch and channel dimensions: {frame_final.shape}")
    
    # Save the processed frame
    cv2.imwrite(save_path, frame_resized * 255)
    
    return frame_final


for img in imgs:
    predicted_labels = predict_image_class(model, img, class_mapping)
    print(f'Predicted Labels {img} -----> {predicted_labels}')

Predicted Labels A_test.jpg -----> ['A']
Predicted Labels B_test.jpg -----> ['B']
Predicted Labels C_test.jpg -----> ['C']
Predicted Labels D_test.jpg -----> ['D']
Predicted Labels E_test.jpg -----> ['E']
Predicted Labels F_test.jpg -----> ['F']
Predicted Labels G_test.jpg -----> ['G']
Predicted Labels H_test.jpg -----> ['H']
Predicted Labels I_test.jpg -----> ['I']
Predicted Labels J_test.jpg -----> ['J']
Predicted Labels K_test.jpg -----> ['K']
Predicted Labels L_test.jpg -----> ['L']
Predicted Labels M_test.jpg -----> ['M']
Predicted Labels nothing_test.jpg -----> ['nothing']
Predicted Labels N_test.jpg -----> ['N']
Predicted Labels O_test.jpg -----> ['O']
Predicted Labels P_test.jpg -----> ['P']
Predicted Labels Q_test.jpg -----> ['Q']
Predicted Labels R_test.jpg -----> ['R']
Predicted Labels space_test.jpg -----> ['space']
Predicted Labels S_test.jpg -----> ['S']
Predicted Labels T_test.jpg -----> ['T']
Predicted Labels U_test.jpg -----> ['U']
Predicted Labels V_test.jpg -----> ['

In [8]:
import os
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# Constants
train_data_dir = 'C:/Users/danie/sign-language-alpha/data/asl_alphabet_train/asl_alphabet_train'
BATCH_SIZE = 128
IMG_SIZE = (64, 64)

# Generate dataset function
def generate_dataset(path):
    data = {'path': [], 'label': []}
    folders = os.listdir(path)
    for folder in folders:
        folderpath = os.path.join(path, folder)
        files = os.listdir(folderpath)
        for file in files:
            filepath = os.path.join(folderpath, file)
            data['path'].append(filepath)
            data['label'].append(folder)
    return pd.DataFrame(data)

# Split dataset function
def dataset_splitter(dataset, train_size=0.9, shuffle=True, random_state=0):
    train_df, val_df = train_test_split(dataset, train_size=train_size, shuffle=shuffle, random_state=random_state)
    train_df = train_df.reset_index(drop=True)
    val_df = val_df.reset_index(drop=True)
    return train_df, val_df

# Generate and split dataset
df = generate_dataset(train_data_dir)
train_df, val_df = dataset_splitter(df)

# Image data generators
train_generator = ImageDataGenerator(rescale=1./255, 
                                     rotation_range=10, 
                                     height_shift_range=0.1, 
                                     width_shift_range=0.1)
val_generator = ImageDataGenerator(rescale=1./255)

train_images = train_generator.flow_from_dataframe(train_df, x_col='path', y_col='label', 
                                                   color_mode='grayscale', class_mode='categorical',
                                                   batch_size=BATCH_SIZE, target_size=IMG_SIZE, 
                                                   shuffle=True, seed=0)

val_images = val_generator.flow_from_dataframe(val_df, x_col='path', y_col='label', 
                                               color_mode='grayscale', class_mode='categorical', 
                                               batch_size=BATCH_SIZE, target_size=IMG_SIZE)

# Load model
model = load_model("FINAL.h5")

# Class mapping
class_mapping = train_images.class_indices

# Get class label from predictions
def get_class_label(predictions, class_mapping):
    labels_mapping = {v: k for k, v in class_mapping.items()}
    predicted_labels = [labels_mapping[pred] for pred in predictions]
    return predicted_labels

# Predict image class
def predict_image_class(model, img, class_mapping):
    if img is None:
        print("Error: Unable to load image.")
        return ["Error"]
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (64, 64))
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=0)
    img = np.expand_dims(img, axis=-1)
    predictions = np.argmax(model.predict(img, verbose=0), axis=1)
    return get_class_label(predictions, class_mapping)

# Process video frame
def process_frame(frame):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame = cv2.resize(frame, (64, 64))
    frame = frame.astype('float32') / 255.0
    frame = np.expand_dims(frame, axis=0)
    frame = np.expand_dims(frame, axis=-1)
    return frame

# Predict image class from frame
def predict_image_class_from_frame(model, frame, class_mapping):
    processed_frame = process_frame(frame)
    predictions = np.argmax(model.predict(processed_frame, verbose=0), axis=1)
    return get_class_label(predictions, class_mapping)

# Main script to capture video and make predictions
cap = cv2.VideoCapture(0)
frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    predicted_labels = predict_image_class_from_frame(model, frame, class_mapping)
    
    # Save the frame for debugging
    frame_filename = f'debug_frame_{frame_count}.png'
    cv2.imwrite(frame_filename, frame)
    
    # Load the saved frame to verify preprocessing
    loaded_frame = cv2.imread(frame_filename)
    if loaded_frame is None:
        print(f"Error: Unable to load saved frame {frame_filename}")
        continue
    
    # Process the loaded frame
    processed_loaded_frame = process_frame(loaded_frame)
    
    # Ensure the loaded frame and original frame preprocessing are consistent
    assert np.array_equal(processed_loaded_frame, process_frame(frame)), "Mismatch in frame preprocessing"

    # Display the predicted label on the frame
    cv2.putText(frame, predicted_labels[0], (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    
    # Display the frame
    cv2.imshow('Sign Language Prediction', frame)
    
    frame_count += 1
    
    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Found 74798 validated image filenames belonging to 29 classes.


C:\Users\danie\anaconda3\envs\myenv\lib\site-packages\keras\src\legacy\preprocessing\image.py:920: UserWarning: Found 1 invalid image filename(s) in x_col="path". These filename(s) will be ignored.
  warnings.warn(


Found 8312 validated image filenames belonging to 29 classes.


In [9]:
def examine_image_shapes(original_frame, saved_frame_path):
    saved_frame = cv2.imread(saved_frame_path)
    
    if saved_frame is None:
        print(f"Error: Unable to load saved frame {saved_frame_path}")
        return
    
    print(f"Original frame shape: {original_frame.shape}")
    print(f"Saved frame shape: {saved_frame.shape}")
    
    processed_original = process_frame(original_frame)
    processed_saved = process_frame(saved_frame)
    
    print(f"Processed original frame shape: {processed_original.shape}")
    print(f"Processed saved frame shape: {processed_saved.shape}")
    
    if np.array_equal(processed_original, processed_saved):
        print("The processed original frame and saved frame are identical.")
    else:
        print("There is a mismatch between the processed original frame and saved frame.")

# Test the shape examination function with an example frame
test_frame = cv2.imread('debug_frame_16.png')  # Example frame captured earlier
examine_image_shapes(frame, 'debug_frame_0.png')

Original frame shape: (480, 640, 3)
Saved frame shape: (480, 640, 3)
Processed original frame shape: (1, 64, 64, 1)
Processed saved frame shape: (1, 64, 64, 1)
There is a mismatch between the processed original frame and saved frame.


In [10]:
imgs_dir = r'C:\Users\danie\sign-language-alpha\data\asl_alphabet_test\asl_alphabet_test'
imgs = os.listdir(imgs_dir)

class_mapping = train_images.class_indices

def get_class_label(predictions, class_mapping):
    labels_mapping = {v: k for k, v in class_mapping.items()}
    predicted_labels = [labels_mapping[pred] for pred in predictions]

    return predicted_labels

def predict_image_class(model, img_path, class_mapping):
    img = cv2.imread(os.path.join(imgs_dir, img_path))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = np.expand_dims(cv2.resize(img, (64, 64)), axis = 0)
    predictions = np.argmax(model.predict(img, verbose = 0), axis = 1)
    predicted_labels = get_class_label(predictions, class_mapping)
    return predicted_labels

import cv2
import numpy as np

def process_and_save_video_frame(frame, save_path="processed_frame.jpg"):
    """
    Process a video frame: convert to grayscale, resize with aspect ratio,
    normalize, reshape, and save the processed frame.
    
    Args:
        frame: The original video frame captured from the webcam.
        save_path: The path to save the processed frame image.
    
    Returns:
        frame_final: The processed frame ready for model input.
    """
    def resize_with_aspect_ratio(image, target_size):
        h, w = image.shape[:2]
        target_w, target_h = target_size
        scale = min(target_w / w, target_h / h)
        new_w, new_h = int(w * scale), int(h * scale)
        resized = cv2.resize(image, (new_w, new_h))
        delta_w, delta_h = target_w - new_w, target_h - new_h
        top, bottom = delta_h // 2, delta_h - (delta_h // 2)
        left, right = delta_w // 2, delta_w - (delta_w // 2)
        color = [0, 0, 0]
        new_image = cv2.copyMakeBorder(resized, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
        return new_image

    print(f"Original Frame shape: {frame.shape}")
    
    # Convert to grayscale
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Resize with aspect ratio
    frame_resized = resize_with_aspect_ratio(frame_gray, (64, 64))
    print(f"Shape after resizing with aspect ratio to 64x64: {frame_resized.shape}")
    
    # Normalize
    frame_normalized = frame_resized.astype('float32') / 255.0
    
    # Add batch and channel dimensions
    frame_final = np.expand_dims(frame_normalized, axis=0)
    frame_final = np.expand_dims(frame_final, axis=-1)
    print(f"Shape after adding batch and channel dimensions: {frame_final.shape}")
    
    # Save the processed frame
    cv2.imwrite(save_path, frame_resized * 255)
    
    return frame_final

img = r'C:\Users\danie\sign-langauge-translator\src\PYTHON_BACKEND\debug_frame_91.png'

predicted_labels = predict_image_class(model, img, class_mapping)
print(f'Predicted Labels {img} -----> {predicted_labels}')


    

Predicted Labels C:\Users\danie\sign-langauge-translator\src\PYTHON_BACKEND\debug_frame_91.png -----> ['O']


In [11]:
import cv2
import numpy as np
import os
from tensorflow.keras.models import load_model

# Load your model
model = load_model('path_to_your_model.h5')

# Assume `train_images` is available and has `class_indices` attribute
class_mapping = train_images.class_indices

def get_class_label(predictions, class_mapping):
    labels_mapping = {v: k for k, v in class_mapping.items()}
    predicted_labels = [labels_mapping[pred] for pred in predictions]
    return predicted_labels

def predict_image_class(model, frame, class_mapping):
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_resized = cv2.resize(frame_gray, (64, 64))
    frame_expanded = np.expand_dims(frame_resized, axis=(0, -1))
    frame_normalized = frame_expanded.astype('float32') / 255.0
    predictions = np.argmax(model.predict(frame_normalized, verbose=0), axis=1)
    predicted_labels = get_class_label(predictions, class_mapping)
    return predicted_labels

def process_and_save_video_frame(frame, save_path="processed_frame.jpg"):
    def resize_with_aspect_ratio(image, target_size):
        h, w = image.shape[:2]
        target_w, target_h = target_size
        scale = min(target_w / w, target_h / h)
        new_w, new_h = int(w * scale), int(h * scale)
        resized = cv2.resize(image, (new_w, new_h))
        delta_w, delta_h = target_w - new_w, target_h - new_h
        top, bottom = delta_h // 2, delta_h - (delta_h // 2)
        left, right = delta_w // 2, delta_w - (delta_w // 2)
        color = [0, 0, 0]
        new_image = cv2.copyMakeBorder(resized, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
        return new_image

    print(f"Original Frame shape: {frame.shape}")
    
    # Convert to grayscale
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Resize with aspect ratio
    frame_resized = resize_with_aspect_ratio(frame_gray, (64, 64))
    print(f"Shape after resizing with aspect ratio to 64x64: {frame_resized.shape}")
    
    # Normalize
    frame_normalized = frame_resized.astype('float32') / 255.0
    
    # Add batch and channel dimensions
    frame_final = np.expand_dims(frame_normalized, axis=0)
    frame_final = np.expand_dims(frame_final, axis=-1)
    print(f"Shape after adding batch and channel dimensions: {frame_final.shape}")
    
    # Save the processed frame
    cv2.imwrite(save_path, frame_resized * 255)
    
    return frame_final

# OpenCV camera capture
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    predicted_labels = predict_image_class(model, frame, class_mapping)
    print(f'Predicted Labels: {predicted_labels}')

    # Display the frame
    cv2.imshow('Frame', frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'path_to_your_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)